In [68]:
# Imports
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [69]:
url = "https://www.ccny.cuny.edu/registrar/fall"
r = requests.get(url)
html_doc = r.text
soup = BeautifulSoup(html_doc)
# print(soup.prettify())

In [70]:
table = soup.find_all('table')[0]
# print(table)

rows = table.find_all('tr')
# print(rows)

In [ ]:
calendar_rows = {
    "date": [],
    "day of the week": [],
    "text": [],
}


# Returns the start date explicitly
def clean_date(date):
    default_year = ", 2021"

    start_date = date.strip().split("-")[0]

    if len(start_date.split(",")) == 1:
        start_date = start_date.strip() + default_year

    return datetime.strptime(start_date, "%B %d, %Y").date()

# Returns the day of the week mentioned
def clean_dotw(dotw):
    return dotw.strip().replace("\n", "").replace("\t", "").split(" ")[0]


def clean_text(text):
    return text.strip().replace("\n", " ").replace("\t", "")


for row in rows:
    row_data = row.find_all("td")

    if row_data:
        calendar_rows["date"].append(clean_date(row_data[0].text))
        calendar_rows["day of the week"].append(clean_dotw(row_data[1].text))
        calendar_rows["text"].append(clean_text(row_data[2].text))

# print(calendar_rows)